In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkregressor.csv')

In [3]:
N = 2021

In [4]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [5]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [6]:
print(*data.columns)

season round podium driver constructor circuit_id driver_points_from qualifying_pos starting_grid stage_q3 driver_points_per constructor_points_per constructor_points_before driver_points_before driver_wins_before constructor_wins_before constructor_standings_before driver_standings_before driver_last_3 constructor_last_3


In [7]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        winners += list(prediction_df.predicted_winner.values)
        points += list(prediction_df.predicted_points.values)
        
        # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)


        # model.fit(X_train, y_train)

    return points, winners

In [8]:
params = parameters.iloc[0]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                           959
model                       nn_regressor
hidden_layer_sizes    (200, 100, 50, 25)
activation                          tanh
solver                              adam
alpha                               10.0
score                           0.789474
Name: 0, dtype: object

In [9]:
test = df[(df.season == N)]

model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, 
    alpha=alpha, 
    random_state=1)
    
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [10]:
len(test.query('predicted_winner == 1 & podium == 1'))/ test['round'].nunique()

0.7368421052631579

In [11]:
df = test[['season', 'round', 'podium', 'predicted_winner','starting_grid','driver_points_from', 'predicted_points']]

In [12]:
df.query('predicted_winner == 1')

,season,round,podium,predicted_winner,starting_grid,driver_points_from,predicted_points
2743,2021,1,2,1,1,18.0,15.184969
2763,2021,2,2,1,1,19.0,17.237805
2781,2021,3,1,1,2,25.0,16.878084
2801,2021,4,1,1,1,25.0,17.700164
2821,2021,5,1,1,2,25.0,16.354196
2853,2021,6,15,1,2,0.0,17.483615
2858,2021,7,1,1,1,26.0,17.440289
2876,2021,8,1,1,1,25.0,17.706486
2896,2021,9,1,1,1,26.0,17.888711
2924,2021,11,9,1,3,2.0,17.702777


In [13]:
df.query('round == 12')

,season,round,podium,predicted_winner,starting_grid,driver_points_from,predicted_points
2935,2021,12,1,1,1,12.0,17.982026
2936,2021,12,2,0,2,9.0,8.846083
2937,2021,12,3,0,3,7.0,16.892240
2938,2021,12,4,0,4,6.0,8.124897
2939,2021,12,5,0,5,5.0,5.172819
2940,2021,12,6,0,6,4.0,5.171913
2941,2021,12,7,0,8,3.0,5.162155
2942,2021,12,8,0,9,2.0,4.908514
2943,2021,12,9,0,10,1.0,1.379671
2944,2021,12,10,0,11,0.0,4.114295
